## Machine learning ensemble modelling of $\textit{Aedes albopictus}$ habitat suitability in the 21$^{21st}$ century
### PLOS ONE 2021

####  Dr. Pantelis Georgiades, Environmental Predictions Department, Climate & Atmosphere Research Centre, The Cyprus Institute  
  
This notebook can be used to download and construct the datasets needed for projecting the Aedes albopictus habitat suitability.

In [ ]:
import os
import xarray as xr
import pandas as pd
import numpy as np
import urllib
import zipfile
import multiprocessing as mp
from functools import partial
import warnings
from sys import argv
import datetime

# Land Use data

### Ref: 
- Harmonization of global land use change and management for the period 850–2100 (LUH2) for CMIP6, Hurtt et al., Geosci. Model Dev., 13, 5425–5464, 2020
https://doi.org/10.5194/gmd-13-5425-2020
- GCAM-Demeter land use dataset at 0.05-degree resolution - https://dx.doi.org/10.25584/data.2020-07.1357/1644253    
  
  Download SSP2 RCP4.5 and SSP5 RCP8.5 GCAM-Harmonized (model means)   
  (Data is freely avaibable but personal account and login needed to download)  

  Files used :  
  GCAM_Demeter_LU_H_ssp2_rcp45_modelmean_2015.nc -> GCAM_Demeter_LU_H_ssp2_rcp45_modelmean_2100.nc  
  GCAM_Demeter_LU_H_ssp5_rcp85_modelmean_2015.nc -> GCAM_Demeter_LU_H_ssp5_rcp85_modelmean_2100.nc  
  

In [ ]:
def get_land_use():
    # Create the folders to hold the data if they don't exist
    if not os.path.isdir('dat'):
        os.mkdir('dat')
    if not os.path.isdir('dat/land_use'):
        os.mkdir('dat/land_use')
    # Download the datasets
    if not os.path.isfile('dat/land_use/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-MESSAGE-ssp245-2-1-f_gn_2015-2100.nc'):
       urllib.request.urlretrieve('http://gsweb1vh2.umd.edu/LUH2/LUH2_v2f/MESSAGE/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-MESSAGE-ssp245-2-1-f_gn_2015-2100.nc',
                                  'dat/land_use/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-MESSAGE-ssp245-2-1-f_gn_2015-2100.nc')
    if not os.path.isfile('dat/land_use/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-MAGPIE-ssp585-2-1-f_gn_2015-2100.nc'):
        urllib.request.urlretrieve('http://gsweb1vh2.umd.edu/LUH2/LUH2_v2f/MAGPIE/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-MAGPIE-ssp585-2-1-f_gn_2015-2100.nc',
                                   'dat/land_use/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-MAGPIE-ssp585-2-1-f_gn_2015-2100.nc')
    # Load the datasets
    if not os.path.isfile('dat/land_use/luh_rcp45_monthly.nc'):
        luh45 = xr\
            .open_dataset('dat/land_use/multiple-states_input4MIPs_landState_ScenarioMIP_' +
                          'UofMD-MESSAGE-ssp245-2-1-f_gn_2015-2100.nc',
                          decode_times=False) \
            .assign(time=pd.to_datetime([str(x) + '-12-31' for x in np.arange(2015, 2101, 1)]))\
            .expand_dims(scenario=['rcp45'])\
            .drop(['lat_bounds', 'lon_bounds', 'time_bnds'])
        # Interpolate to a monthly temporal resolution
        luh45 = luh45.interp(time = pd.date_range(start='2015-01-31', periods = 86*12, freq='M'),
                             kwargs = {'fill_value': 'extrapolate'},
                             method='nearest')
        luh45.to_netcdf('dat/land_use/luh_rcp45_monthly.nc')
    if not os.path.isfile('dat/luh_rcp85_monthly.nc'):
        luh85 = xr \
            .open_dataset('dat/land_use/multiple-states_input4MIPs_landState_ScenarioMIP' +
                          '_UofMD-MAGPIE-ssp585-2-1-f_gn_2015-2100.nc',
                          decode_times=False) \
            .assign(time=pd.to_datetime([str(x) + '-12-31' for x in np.arange(2015, 2101, 1)]))\
            .expand_dims(scenario=['rcp85'])\
            .drop(['lat_bounds', 'lon_bounds', 'time_bnds'])
        # Interpolate to a monthly temporal resolution
        luh85 = luh85.interp(time = pd.date_range(start='2015-01-31', periods = 86*12, freq='M'),
                             kwargs = {'fill_value': 'extrapolate'},
                             method='nearest')
        luh85.to_netcdf('dat/land_use/luh_rcp85_monthly.nc')
        
        
def gcam_land_use():
    path_gcam = 'dat/land_use_GCAM'
    if not os.path.isfile(f'{path_gcam}/gcam.nc'):
        files = sorted(os.listdir(path_gcam))
        # Longitude and latitude ranges in the population density datasets
        lon = np.arange(-179.875, 180, 0.25)
        lat = np.arange(-89.875, 90, 0.25)
        for file in files:
            print(file)
            scenario = file.split('_')[5]
            year = file.split('_')[-1].split('.')[0]
            file_temp = xr\
                .open_dataset(f'{path_gcam}/{file}')\
                .expand_dims(time=pd.to_datetime([f'{year}-01-31']), 
                             scenario=[scenario])\
                .rename({'longitude': 'lon', 'latitude': 'lat'})
            # Interpolate to 0.25 degrees regular grid
            file_temp = file_temp.interp(lon=lon, lat=lat, method='nearest')
            if file == files[0]:
                ds_gcam = file_temp
            else:
                ds_gcam = ds_gcam.merge(file_temp)
            del file_temp, file
        ds_gcam.to_netcdf(f'{path_gcam}/gcam.nc')
    else:
        ds_gcam = xr.open_dataset(f'{path_gcam}/gcam.nc')
    # Interpolate to a monthly temporal resolution (each scenario separately to conserve memory
    if not os.path.isfile(f'{path_gcam}/gcam_monthly_rcp45.nc'):
        print('Interpolating RCP4.5 dataset')
        gcam45 = ds_gcam.sel(scenario = 'rcp45', lat=slice(-54, 84)).drop('scenario')
        gcam45 = gcam45.interp(time = pd.date_range('2015-01-31', periods = 86*12, freq='M'),
                               kwargs = {'fill_value': 'extrapolate'},
                               method = 'nearest')
        print('Saving')
        gcam45.to_netcdf(f'{path_gcam}/gcam_monthly_rcp45.nc')
        del gcam45
    if not os.path.isfile(f'{path_gcam}/gcam_monthly_rcp85.nc'):
        print('Interpolating RCP8.5 dataset')
        gcam85 = ds_gcam.sel(scenario = 'rcp85', lat=slice(-54, 84)).drop('scenario')
        del ds_gcam
        gcam85 = gcam85.interp(time = pd.date_range('2015-01-31', periods = 86*12, freq='M'),
                               kwargs = {'fill_value': 'extrapolate'},
                               method = 'nearest')
        print('Saving')
        gcam85.to_netcdf(f'{path_gcam}/gcam_monthly_rcp85.nc')
        del gcam85

# Population Density

### Ref: 

Spatially explicit global population scenarios consistent with the Shared Socioeconomic Pathways, B Jones and B C O'Neill 2016 Environ. Res. Lett. 11 084003

https://www.cgd.ucar.edu/iam/modeling/spatial-population-scenarios.html

In [ ]:
def get_pop_density():
    # Create the folders to hold the data if they don't exist
    if not os.path.isdir('dat'):
        os.mkdir('dat')
    if not os.path.isdir('dat/pop_density'):
        os.mkdir('dat/pop_density')
    if not os.path.isfile('dat/pop_density/pop_dens_rcp45.nc'):
        # Download the datasets
        if not os.path.isfile('Spatial_population_scenarios_NetCDF.zip'):
            urllib.request.urlretrieve('https://www.cgd.ucar.edu/iam/modeling/data/Spatial_population_scenarios_NetCDF.zip',
                                       'dat/pop_density/Spatial_population_scenarios_NetCDF.zip')
            # Extract
            with zipfile.ZipFile('dat/pop_density/Spatial_population_scenarios_NetCDF.zip') as zip_file:
                zip_file.extractall('dat/pop_density')
        # Read the datasets
        path_pop_rcp45 = 'dat/pop_density/NetCDF/SSP2_NetCDF/total/NetCDF/'
        path_pop_rcp85 = 'dat/pop_density/NetCDF/SSP5_NetCDF/total/NetCDF/'
        # List the datasets in the two folders
        files45 = sorted(os.listdir(path_pop_rcp45))
        files85 = sorted(os.listdir(path_pop_rcp85))
        # Longitude and latitude ranges in the population density datasets
        lon = np.arange(-179.875, 180, 0.25)
        lat = np.arange(-54.875, 84, 0.25)
        for i in np.arange(len(files45)):
            pop45 = xr\
                .open_dataset(f'{path_pop_rcp45}/{files45[i]}')\
                .expand_dims(time=pd.to_datetime([files45[i].split('_')[-1].split('.')[0] + '-01-31']))\
                .rename_vars({files45[i].replace('.nc', ''): 'pop_density'})
            pop85 = xr\
                .open_dataset(f'{path_pop_rcp85}/{files85[i]}')\
                .expand_dims(time=pd.to_datetime([files85[i].split('_')[-1].split('.')[0] + '-01-31']))\
                .rename_vars({files85[i].replace('.nc', ''): 'pop_density'})
            # Interpolate to 0.25 regular grid
            pop45 = pop45.interp(lon=lon, lat=lat, method='nearest')
            pop85 = pop85.interp(lon=lon, lat=lat, method='nearest')
            if i == 0:
                pop_dens_45 = pop45
                pop_dens_85 = pop85
            else:
                pop_dens_45 = pop_dens_45.merge(pop45)
                pop_dens_85 = pop_dens_85.merge(pop85)
            del pop45, pop85
        # Interpolate in time to monthly
        pop_dens_45 = pop_dens_45.interp(time=pd.date_range(start='2010-01-31', periods=91*12, freq='M'), 
                                         kwargs={'fill_value': 'extrapolate'},
                                         method='nearest')
        pop_dens_85 = pop_dens_85.interp(time=pd.date_range(start='2010-01-31', periods=91*12, freq='M'), 
                                         kwargs={'fill_value': 'extrapolate'},
                                         method='nearest')
        pop_dens_45.to_netcdf('dat/pop_density/pop_dens_rcp45.nc')
        pop_dens_85.to_netcdf('dat/pop_density/pop_dens_rcp85.nc')

# NASA-NEX-GDDP
## NASA Earth Exchange Global Daily Downscaled Climate Projections

The provided function get_year() takes an xarray of NASA-NEX_GDDP data for a year, with daily temporal resolution and 0.25 degree spatial resolution. The xarrays variables are pr, tas, tasmin, tasmax (Precipitation, Mean temperature, Min temperature, Max temperature). It will aggregate the data to a monthly temporal resolution and calculate all the biologically relevant features used in the study.

Ref:  
Thrasher, B., Maurer, E. P., McKellar, C., & Duffy, P. B., 2012: Technical Note: Bias correcting climate model simulated daily temperature extremes with quantile mapping. Hydrology and Earth System Sciences, 16(9), 3309-3314. doi:10.5194/hess-16-3309-2012

In [ ]:
# Get the data for a year
def get_year(ds):
    # Resample the data monthly
    ds = ds.resample(time='M').mean()
    # Land/sea mask data
    lsmask = xr\
        .open_dataset('dat/lsmask.nc')
    # Add to the dataset
    ds = ds.merge(lsmask)
    del lsmask
    # Convert the dataset to dataframe and select the land
    df = ds.to_dataframe().reset_index(drop=False)
    df = df.loc[df.lsmask == 0]
    # Bio 1—Annual Mean Temperature
    df_bio = df\
        .groupby(['lat', 'lon'], as_index=False)\
        .agg({'tas': np.mean})\
        .rename(columns={'tas': 'bio1'})
    # Bio 2—Annual Mean Diurnal Range
    df = df.assign(diurnal=df.tasmax-df.tasmin)
    df_bio = pd.merge(df_bio, df.groupby(['lat', 'lon'], as_index=False)
                      .agg({'diurnal': np.mean})
                      .rename(columns={'diurnal': 'bio2'}),
                      on=['lat', 'lon'], how='left')
    # Bio 4—Temperature Seasonality(Standard Deviation)
    # Bio 5—Max Temperature of Warmest Month
    # Bio 6—Min Temperature of Coldest Month
    df_bio = pd.merge(df_bio, df.groupby(['lat', 'lon'], as_index=False)
                      .agg({'tas': np.std, 'tasmax': np.max, 'tasmin': np.min})
                      .rename(columns={'tas': 'bio4', 'tasmax': 'bio5', 'tasmin': 'bio6'}),
                      on=['lat', 'lon'], how='left')
    # Bio 4a—Temperature Seasonality (CV)
    df_bio = df_bio.assign(bio4a=100*(df_bio.bio4/df_bio.bio1))
    # Bio 7—Annual Temperature Range
    df_bio = df_bio.assign(bio7=df_bio.bio5-df_bio.bio6)
    # Bio 8-11, 16-19
    coords = df_bio[['lat', 'lon']].values
    pool = mp.Pool(120)
    func = partial(bio8_calc, df=df)
    bio8to11 = pd.concat(pool.map(func, coords))
    pool.close()
    pool.join()
    bio8to11.reset_index(drop=True, inplace=True)
    # Add to the rest
    df_bio = pd.merge(df_bio, bio8to11, on=['lat', 'lon'], how='left')
    del bio8to11, func
    # Bio 12—Annual Precipitation
    # Bio 13—Precipitation of Wettest Month
    # Bio 14—Precipitation of Driest Month
    df_temp = df.groupby(['lat', 'lon'])\
        .agg({'pr': [np.sum, np.max, np.min]})\
        .reset_index(drop=False, level=[0, 1])
    df_temp.columns = [''.join(x) for x in df_temp.columns.values]
    df_temp.rename(columns={'prsum': 'bio12', 'pramax': 'bio13', 'pramin': 'bio14'},
                   inplace=True)
    df_bio = pd.merge(df_bio, df_temp, on=['lat', 'lon'], how='left')
    # Bio 15—Precipitation Seasonality (CV)
    df_temp = df.groupby(['lat', 'lon'], as_index=False)\
        .agg({'pr': np.std})\
        .rename(columns={'pr': 'pr_std'})
    df_temp = pd.merge(df_temp, df_bio[['lat', 'lon', 'bio12']],
                       on=['lat', 'lon'], how='left')
    df_temp = df_temp.assign(bio15=100*(df_temp.pr_std/(1+(df_temp.bio12/12))))
    df_bio = pd.merge(df_bio, df_temp[['lat', 'lon', 'bio15']],
                      on=['lat', 'lon'], how='left')
    # Bio 3—Isothermality
    df_bio = df_bio.assign(bio3=100*(df_bio.bio2/df_bio.bio7))
    # Get all coordinates
    coords = get_coords(sorted(df.time.unique()))
    df_fin = pd.merge(coords, pd.merge(df, df_bio, on=['lat', 'lon'], how='left'),
                      on=['lat', 'lon', 'time'], how='left')
    ds_fin = df_fin.set_index(['lat', 'lon', 'time']).to_xarray()
    return ds_fin


def bio8_calc(coord, df):
    # Get the data for this coordinates
    df_temp = df\
        .loc[(df.lat == coord[0]) & (df.lon == coord[1])]\
        .reset_index(drop=True)\
        .assign(month=np.arange(1, 13, 1))
    # Get the total precipitation for each quarter
    df_q = pd.DataFrame()
    for q in np.arange(1, 11, 1):
        start = q
        end = q+2
        total_prec = df_temp\
            .loc[(df_temp.month >= start) & (df_temp.month <= end)].pr.sum()
        total_tas = df_temp \
            .loc[(df_temp.month >= start) & (df_temp.month <= end)].tas.sum()
        df_q = df_q.append(pd.DataFrame({'month_start': [start],
                                         'month_end': [end],
                                         'total_pr': [total_prec],
                                         'total_tas': [total_tas]}))
    # Get the mean temperature and total precipitation of the quarter
    # with the highest precipitation
    start, end = df_q\
        .loc[df_q.total_pr == df_q.total_pr.max()][['month_start', 'month_end']]\
        .values[0]
    temp_q = df_temp\
        .loc[(df_temp.month >= start) & (df_temp.month <= end)]\
        .tas\
        .mean()
    pr_high = df_temp\
        .loc[(df_temp.month >= start) & (df_temp.month <= end)]\
        .pr.sum()
    # Get the mean temperature and total precipiation of the quarter
    # with the lowest precipitation
    start, end = df_q \
        .loc[df_q.total_pr == df_q.total_pr.min()][['month_start', 'month_end']] \
        .values[0]
    temp_q_low = df_temp \
        .loc[(df_temp.month >= start) & (df_temp.month <= end)] \
        .tas \
        .mean()
    pr_low = df_temp \
        .loc[(df_temp.month >= start) & (df_temp.month <= end)] \
        .pr.sum()
    # Get the mean temperature of the hottest quarter
    start, end = df_q \
        .loc[df_q.total_tas == df_q.total_tas.max()][['month_start', 'month_end']] \
        .values[0]
    temp_max = df_temp \
        .loc[(df_temp.month >= start) & (df_temp.month <= end)] \
        .tas \
        .mean()
    pr_temp_high = df_temp \
        .loc[(df_temp.month >= start) & (df_temp.month <= end)] \
        .pr.sum()
    # Get the mean temperature of the coldest quarter
    start, end = df_q \
        .loc[df_q.total_tas == df_q.total_tas.min()][['month_start', 'month_end']] \
        .values[0]
    temp_low = df_temp \
        .loc[(df_temp.month >= start) & (df_temp.month <= end)] \
        .tas \
        .mean()
    pr_temp_low = df_temp \
        .loc[(df_temp.month >= start) & (df_temp.month <= end)] \
        .pr.sum()
    df_ret = pd.DataFrame({'lat': [coord[0]], 'lon': [coord[1]],
                           'bio8': [temp_q], 'bio9': [temp_q_low],
                           'bio10': [temp_max], 'bio11': [temp_low],
                           'bio16': [pr_high], 'bio17': [pr_low],
                           'bio18': [pr_temp_high], 'bio19': [pr_temp_low]})
    return df_ret


# Function to construct the coordinates for all globe
def get_coords(dates):
    lons = []
    lats = []
    time = []
    for lon in np.arange(-179.875, 180, 0.25):
        for lat in np.arange(-89.875, 90, 0.25):
            for date in dates:
                lons.append(lon)
                lats.append(lat)
                time.append(date)
    return pd.DataFrame({'lat': lats, 'lon': lons, 'time': time})

### The following script can be used to construct a dataset for a specified climate model, year and scenario for predicting Aedes albopictus habitat suitability using the ensebmle machine learning model.

An example dataset for the CNRM-CM5 climate model, scenario RCP8.5 for the year 2050 is provided in the dat folder.

NOTE: The year is provided as YYYY (eg. 2021) and the scenario as 'rcp45' or 'rcp85'. The procedure for the climate and biologically relevant feature sets should be modified to reflect the available models downloaded.

In [ ]:
def predict_year_dataset(model, year, scenario='rcp45'):
    # Read the LHU and GCAM land use datasets and select the year specified
    ds_luh = xr.open_dataset(f'dat/land_use/luh_{scenario}_monthly.nc').sel(time=slice(f'{year}-01-31', f'{year}-12-31'))
    ds_gcam = xr.open_dataset(f'dat/land_use_GCAM/gcam_monthly_{scenario}.nc').sel(time=slice(f'{year}-01-31', f'{year}-12-31'))
    # Merge the two
    ds = ds_gcam.merge(ds_luh)
    ds = ds.drop('scenario')
    del ds_luh, ds_gcam
    # Read the population density dataset
    ds_pop = xr.open_dataset(f'dat/pop_density/pop_dens_{scenario}.nc').sel(time=slice(f'{year}-01-31', f'{year}-12-31'))
    # Add to the land use dataset
    ds = ds.merge(ds_pop)
    del ds_pop
    # Read the climate and biovariables dataset
    ds_clim = xr.open_dataset('dat/CCSM4_rcp85_2021.nc')  # <<-- This should be modified to reflect the location of the processed datasets
    # Convert time variable to pandas datetime object
    ds_clim = ds_clim.assign(time=pd.to_datetime([str(x).split(' ')[0] for x in ds_clim.time.values]))
    # Merge with the rest
    ds = ds.merge(ds_clim)
    del ds_clim
    # Add the land/sea mask (not used in the model, but used in the prediction procedures)
    ds = ds.merge(xr.open_dataset('dat/lsmask.nc'))
    return ds

In [ ]:
ds = predict_year_dataset(model='CNRM5-CM5', year=2050, scenario='rcp85')
ds